In [1]:
import pickle
import nltk
import math

In [44]:
class vect:
    SCALAR = 1
    COSINE = 2
    JACCARD = 3

class Docs:

    def __init__(self,Lancaster=True,split=False):

        docs = []
        self.maxfreq = {}
        MotsVides = nltk.corpus.stopwords.words('english')
        self.Lancaster = Lancaster
        self.split = split
        self.files = ["TP2data/D1.txt","TP2data/D2.txt","TP2data/D3.txt","TP2data/D4.txt","TP2data/D5.txt","TP2data/D6.txt"]
        file = ""
        my_dict = {}
        for i in range(len(self.files)):
            file = open(self.files[i]).read()
            file = file.replace("\n","")

            if not split :
                Termes = file.split()
            if split:

                ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 


            if Lancaster :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                porter = nltk.PorterStemmer()
                TermesNormalisation = [porter.stem(terme) for terme in TermesSansMotsVides]


            for term in TermesNormalisation:
                
                if (term,i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict


        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if keys[1] not in self.maxfreq.keys():
                self.maxfreq[keys[1]] = 0
            if self.dict[keys] > self.maxfreq[keys[1]]:
                self.maxfreq[keys[1]] = self.dict[keys]

        self.N = len(docs)
    
    def rebuild(self,Lancaster=True,split=False):
        docs = []
        self.maxfreq = {}
        MotsVides = nltk.corpus.stopwords.words('english')
        self.Lancaster = Lancaster
        self.split = split
        
        file = ""
        my_dict = {}
        for i in range(len(self.files)):
            file = open(self.files[i]).read()
            file = file.replace("\n","")


            if not split :
                Termes = file.split()
            if split:
                ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>> 
                Termes = ExpReg.tokenize(file) 


            if Lancaster :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                Lancaster = nltk.LancasterStemmer()
                TermesNormalisation = [Lancaster.stem(terme) for terme in TermesSansMotsVides]
            else :
                TermesSansMotsVides = [terme for terme in Termes if terme.lower() not in MotsVides]
                porter = nltk.PorterStemmer()
                TermesNormalisation = [porter.stem(terme) for terme in TermesSansMotsVides]

            for term in TermesNormalisation:
                
                if (term.lower(),i) in my_dict:
                    my_dict[(term,i)] += 1
                else:
                    my_dict[(term,i)] = 1

        self.dict = my_dict


        for keys in self.dict.keys():
            if keys[1] not in docs:
                docs.append(keys[1])
            if keys[1] not in self.maxfreq.keys():
                self.maxfreq[keys[1]] = 0
            if self.dict[keys] > self.maxfreq[keys[1]]:

                self.maxfreq[keys[1]] = self.dict[keys]

        self.N = len(docs)
        self.DictWeight()
    def normalize(self,text):
        if self.Lancaster :
            Lancaster = nltk.LancasterStemmer()
            text = Lancaster.stem(text)
        else :
            porter = nltk.PorterStemmer()
            text = porter.stem(text)
        return text
    def splitter(self,text):
        if self.split:
            return text.split(" ")
        else:
            ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>> 
            Termes = ExpReg.tokenize(text) 
            return Termes
    def calcWeight(self,word,doc):

        inDocs = []

        for keys in self.dict.keys():
            if keys[0] == word:
                if keys[1] not in inDocs:
                    inDocs.append(keys[1])

        # nombre de documents contenant le mot
        Ni = len(inDocs)

        try:
            tf = self.dict[(word,doc)]/self.maxfreq[doc]
        except:
            tf = 0
            
        #idf = math.log10((N/Ni)+1)
        try:
            idf = math.log10((self.N/Ni)+1)
        except:
            idf = 0

        return(tf*idf)

    def DictWeight(self):

        self.TDFP = [(word,doc,self.dict[(word,doc)],self.calcWeight(word,doc)) for word,doc in self.dict.keys()]
        self.DTFP = [(doc,word,freq,weight) for word,doc,freq,weight in self.TDFP]
        self.TDFP = sorted(self.TDFP, key=lambda x: x[0])

    def MV(self,termes,typeV = vect.SCALAR):
        rsv = {}
        termes = self.splitter(termes)
        normterms = []
        for term in termes:
            normterms.append(self.normalize(term))

        for doc in range(self.N):
            QD = 0
            v2 = 0
            w2 = 0
            for term in normterms:
                if (term.lower(),doc) in self.dict.keys():
                    weight = self.calcWeight(term,doc)
                    QD+= weight
                    w2+= math.pow(weight,2)
                    v2+=1
            print(w2,v2)
            try:
                if typeV == vect.COSINE:
                    QD = QD / (v2*w2)
                if typeV == vect.JACCARD:
                    QD = QD / (w2+v2 - QD)
            except ZeroDivisionError:
                QD = 0

            rsv[doc] = QD
            
        return rsv

        
d = Docs(Lancaster=False,split=True)

d.DictWeight()


<>:26: SyntaxWarning: invalid escape sequence '\.'
<>:77: SyntaxWarning: invalid escape sequence '\.'
<>:123: SyntaxWarning: invalid escape sequence '\.'
<>:26: SyntaxWarning: invalid escape sequence '\.'
<>:77: SyntaxWarning: invalid escape sequence '\.'
<>:123: SyntaxWarning: invalid escape sequence '\.'
C:\Users\mekki\AppData\Local\Temp\ipykernel_7340\1234238284.py:26: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>>
C:\Users\mekki\AppData\Local\Temp\ipykernel_7340\1234238284.py:77: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Za-z]\.)+|[A-Za-z]+[\-@]\d+(?:\.\d+)?|\d+[A-Za-z]+|\d+(?:[\.\,]\d+)?%?|\w+(?:[\-/]\w+)*') # \d : équivalent à [0-9] >>>
C:\Users\mekki\AppData\Local\Temp\ipykernel_7340\1234238284.py:123: SyntaxWarning: invalid escape sequence '\.'
  ExpReg = nltk.RegexpTokenizer('(?:[A-Z

In [51]:
d.MV("documents ranking",typeV=vect.JACCARD)

0 0
0.08977205194893534 1
0.01896306064961095 2
0.10158604251801275 2
0.08137635711861467 2
0.009111059267549354 2


{0: 0,
 1: 0.3791925699343096,
 2: 0.09677848750518504,
 3: 0.22471246845697213,
 4: 0.20460462614032912,
 5: 0.07181190846697223}